# Set realsense camera `with ROS`

##### you should run the below code before Run this code.

In [ ]:
!roslaunch realsense2_camera rs_d435_camera_with_model.launch default:=temporal,decimation,spatial,disparity

... logging to /home/rilab/.ros/log/82886e3e-a527-11ed-8491-79820dda6b6e/roslaunch-rilab-XPS-15-9500-41270.log
Checking log directory for disk usage. This may take a while.
Press Ctrl-C to interrupt
Done checking log file disk usage. Usage is <1GB.
]2;/home/rilab/catkin_qd/src/realsense-ros/realsense2_camera/launch/rs_d435_camera_with_model.launch
xacro: in-order processing became default in ROS Melodic. You can drop the option.
started roslaunch server http://rilab-XPS-15-9500:45883/

SUMMARY

PARAMETERS
 * /camera/realsense2_camera/accel_fps: 250
 * /camera/realsense2_camera/accel_frame_id: camera_accel_frame
 * /camera/realsense2_camera/accel_optical_frame_id: camera_accel_opti...
 * /camera/realsense2_camera/align_depth: False
 * /camera/realsense2_camera/aligned_depth_to_color_frame_id: camera_aligned_de...
 * /camera/realsense2_camera/aligned_depth_to_fisheye1_frame_id: camera_aligned_de...
 * /camera/realsense2_camera/aligned_depth_to_fisheye2_frame_id: camera_aligned_de...
 *

##### Import libraries

In [ ]:
import sys 
sys.path.append("..")
import matplotlib.pyplot as plt 
from utils.utils_pcl import *
from utils.utils_pointcloud import *
import cv2 
import rospy 
from get_rostopic import RealsenseD435i

# from roslib import message
print("Done.")

Done.


In [ ]:
def plot_rgb_imgae(msg_image, name):
    bridge = cv_bridge.CvBridge()
    img_shape = (640, 480)
    try:
        cv_image_array = bridge.imgmsg_to_cv2(msg_image, "32FC3")
        cv_image_array = np.array(cv_image_array, dtype = np.dtype('f8'))
        np.save("./data/npy/{}.npy".format(name), cv_image_array)
        cv2.imwrite('./data/png/{}.png'.format(name), cv_image_array*1)
        plt.figure(figsize=(10,10))
        for i in range(3):
            plt.subplot(1, 3, i+1)
            plt.imshow(cv_image_array[:,:,i])
        plt.show()

        cv_image_array = cv2.resize(cv_image_array, img_shape, interpolation = cv2.INTER_CUBIC)
        cv_image_array = cv2.normalize(cv_image_array, cv_image_array, 0, 255, cv2.NORM_MINMAX)
        np.save("./realworld_data/npy/{}_raw.npy".format(name), cv_image_array.astype(np.float32))
        cv2.imwrite('./realworld_data/png/{}_raw.png'.format(name), cv_image_array)
        plt.figure(figsize=(10,10))
        for i in range(3):
            plt.subplot(1, 3, i+1)
            plt.imshow(cv_image_array[:,:,i])
        plt.show()

        return cv_image_array

    except cv_bridge.CvBridgeError as e:
        print(e) 


In [ ]:
rospy.init_node('Realsense')
# mode     = "pointcloud"
# mode     = "depth"
mode     = "rgb"

camera   = RealsenseD435i(mode=mode)
name = "230205_test1"

if mode=="depth":
    save_depth_img(msg_depth = camera.depth_image, name = name) 
    np_depth = ros_to_numpy(camera.depth_image)
    print("Shape: {}".format(np_depth.shape))
elif mode == "rgb":
    img_rgb = plot_rgb_imgae(msg_depth = camera.rgb_image, name = name) 
else: 
    save_pc(msg_pc=camera.point_cloud, name=name)
    np_point = ros_to_numpy(camera.point_cloud)
    # print(camera.point_cloud)
    print("Shape: {}".format(np_point.shape))
